<a href="https://colab.research.google.com/github/fralfaro/MAT281_2023/blob/main/docs/lectures/data_manipulation/pd_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pivot & Melt

## Formato Wide y Formato Long

<img src="https://drive.google.com/uc?export=view&id=1NuAyGD-XH0MZd0YI01mBIBSOxski34k8" width = "600" align="center"/>



Dentro del mundo de los dataframe (o datos tabulares) existen dos formas de presentar la naturaleza de los datos: formato wide y formato long. 


Por ejemplo, el conjunto de datos [Zoo Data Set](http://archive.ics.uci.edu/ml/datasets/zoo) presenta las características de diversos animales, de los cuales presentamos las primeras 5 columnas.

|animal_name|hair|feathers|eggs|milk|
|-----------|----|--------|----|----|
|antelope|1|0|0|1|
|bear|1|0|0|1|
|buffalo|1|0|0|1|
|catfish|0|0|1|0|

La tabla así presentada se encuentra en **wide format**, es decir, donde los valores se extienden a través de las columnas.

Sería posible representar el mismo contenido anterior en **long format**, es decir, donde los mismos valores se indicaran a través de las filas:

|animal_name|characteristic|value|
|-----------|----|--------|
|antelope|hair |1|
|antelope|feathers|0|
|antelope|eggs|0|
|antelope|milk|1|
|...|...|...|...|..|
|catfish|hair |0|
|catfish|feathers|0|
|catfish|eggs|1|
|catfish|milk|0|

En python existen maneras de pasar del formato **wide** al formato **long** y viceversa.

## Pivotear y despivotear tablas

### Pivot

El pivoteo de una tabla corresponde al paso de una tabla desde el formato **long** al formato **wide**. Típicamente esto se realiza para poder comparar los valores que se obtienen para algún registro en particular, o para utilizar algunas herramientas de visualización básica que requieren dicho formato.

Para ejemplificar estos resultados, ocupemos el conjunto de datos **terremotos.csv**, con contiene los registros de  terremotos de distintos paises desde el año 2000 al 2011.

<img src="https://drive.google.com/uc?export=view&id=1ZPr9CE2Oq7a5HpaLuqbp43vpaSfPM9ty" width = "200" align="center"/>



In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
# formato long

url='https://drive.google.com/file/d/1Knut8vY4wuDFrO3bEjlQVUwq-CaRNJpu/view?usp=drive_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]

df = pd.read_csv(url, sep=",")
df.head()

Por ejemplo,  se quiere saber el terremoto de mayor magnitud a nivel de país año. Tenemos dos formas de mostrar la información.

In [ ]:
# formato long
df.groupby(['Pais','Año']).max()

In [ ]:
# formato wide
df_pivot = df.pivot_table(index="Pais", columns="Año", values="Magnitud", fill_value='', aggfunc=np.max)
df_pivot

### Despivotear un tabla

El despivotear una tabla corresponde al paso de una tabla desde el formato **wide** al formato **long**. 

Se reconocen dos situaciones:

1. El valor indicado para la columna es único, y sólo se requiere definir correctamente las columnas.
2. El valor indicado por la columna no es único o requiere un procesamiento adicional, y se requiere una iteración más profunda.

Para ejemplificar esto, crearemos un conjunto de datos con los horarios de los ramos que se tiene que dictar en un determinado día, hora y lugar.

<img src="https://drive.google.com/uc?export=view&id=1VpJA_7QwTbpogVnIWMr0mxwSStSaZiCd" width = "250" align="center"/>



**a) El valor indicado para la columna es único**

In [ ]:
columns = ["sala","dia","08:00","09:00","10:00"]
data = [
        ["C201","Lu", "mat1","mat1",    ""],
        ["C201","Ma", "","",""],
        ["C202","Lu", "","",""],
        ["C202","Ma", "mat1","mat1",    ""],
        ["C203","Lu", "fis1","fis1","fis1"],
        ["C203","Ma", "fis1","fis1","fis1"],
       ]
df = pd.DataFrame(data=data, columns=columns)
df

In [ ]:
# Despivotear incorrectamente la tabla
df_melt = df.melt(id_vars=["sala"], var_name="hora", value_name="curso")
df_melt

In [ ]:
# Despivotear correctamente la tabla
df_melt = df.melt(id_vars=["sala", "dia"], var_name="hora", value_name="curso")
df_melt[df_melt.curso!=""].sort_values(["sala","dia","hora"])

**b) Relaciones no únicas**

In [ ]:
columns = ["sala","curso","Lu","Ma","hora"]
data = [
        ["C201","mat1","X","","8:00-10:00"],
        ["C202","mat1","","X","8:00-10:00"],
        ["C203","fis1","X","X","8:00-11:00"],
       ]
df = pd.DataFrame(data=data, columns=columns)
df

#### Métodos

**método 01:  Despivotear manualmente y generar un nuevo dataframe**

* **Ventajas**: Si se puede es una solución directa y rápida.
* **Desventaja**: requiere programación explícita de la tarea, no es reutilizable.

In [ ]:
# Obtener el día lunes
df_Lu = df.loc[df.Lu=="X", ["sala","curso","hora"]]
df_Lu["dia"] = "Lu"
df_Lu

In [ ]:
# Obtener el día martes
df_Ma = df.loc[df.Ma=="X", ["sala","curso","hora"]]
df_Ma["dia"] = "Ma"
df_Ma

In [ ]:
# Juntar
pd.concat([df_Lu,df_Ma])

**método 02: Iterar sobre las filas y generar contenido para un nuevo dataframe**
* **Ventajas**: En general, fácil de codificar.
* **Desventaja**: puede ser lento, es ineficiente.

In [ ]:
my_columns = ["sala","curso","dia","hora"]
my_data = []
for i, df_row in df.iterrows():
    # Procesar cada fila
    if df_row.Lu=="X":
        my_row = [df_row.sala, df_row.curso, "Lu", df_row.hora]
        my_data.append(my_row)
    if df_row.Ma=="X":
        my_row = [df_row.sala, df_row.curso, "Ma", df_row.hora]
        my_data.append(my_row)
new_df = pd.DataFrame(data=my_data, columns=my_columns)
new_df

## Referencias

1. [Reshaping and pivot tables](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html)